In [1]:
import requests
import pandas as pd
import json

#!pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env into environment
import time
time.sleep(20)

In [2]:

#city data
url="Where you saved your data"
path = "https://drive.google.com/uc?export=download&id="+url.split("/")[-2]
city_df = pd.read_csv(path,low_memory=False)
city_df = city_df.drop(axis = 0, columns = 'Unnamed: 0')

In [3]:
city_df

,city,lat,lon,country_code
0,Berlin,52.517037,13.388860,DE
1,Dortmund,51.514227,7.465279,DE
2,Kassel,51.315455,9.492410,DE
3,Munich,48.137108,11.575382,DE
4,Madrid,40.416705,-3.703582,ES
5,Bilbao,43.263002,-2.935004,ES
6,Seville,37.388630,-5.995340,ES
7,Valencia,39.469707,-0.376335,ES
8,Oslo,59.913330,10.738970,NO
9,Trondheim,63.430566,10.395193,NO


We used visual crossing data free version but then changed to a paid version because of the limiting restrictions of how much data we were able to get.

In [5]:
daily_climate_info = []  # to collect DataFrames for all cities
API_KEY = "YOUR OWN API KEY "
base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"

for index, row in city_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    city_name = row['city']

    url = f"{base_url}/{lat},{lon}/2010-01-01/2024-12-31"
    params = {
        'key': API_KEY,
        'unitGroup': 'metric',
        'include': 'days',
        'elements': 'datetime,tempmax,tempmin,temp,humidity,precip,windspeed,cloudcover,conditions',
        'lang': 'en',
    }

    response = requests.get(url, params=params)
    if response.status_code != 200 :
        print(f"Error {response.status_code} for {city_name} ({lat}, {lon})")
        continue

    data = response.json()
    daily_data = data.get('days', [])

    # Create DataFrame from daily data
    df = pd.DataFrame(daily_data)

    # Add city info columns
    df['city'] = city_name
    df['lat'] = lat
    df['lon'] = lon

    # Convert datetime to pandas datetime type (optional)
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Collect this city's data
    daily_climate_info.append(df)

# After the loop, combine all city DataFrames into one
all_cities_df = pd.concat(daily_climate_info, ignore_index=True)

print(all_cities_df.head())

KeyboardInterrupt: 

In [ ]:
all_cities_df = pd.DataFrame(all_cities_df)
all_cities_df
all_cities_df['city'].unique()

array(['Berlin', 'Dortmund', 'Kassel', 'Munich', 'Madrid', 'Bilbao',
       'Seville', 'Valencia', 'Oslo', 'Trondheim', 'Harstad', 'Alta',
       'Bratislava', 'Košice', 'Žilina'], dtype=object)

In [ ]:
all_cities_df.to_csv('all_cities_df.csv')



Move to juptyer labs to work locally and work with MySQL